In [37]:
import string
import math
import random

In [38]:
## Location data = (lat, long)
DEPOT_LOC = (4.4184,114.0932)

# List to store customer info 
CUSTOMERS = []
# List to store vehicle info
VEHICLES = []

In [39]:
# Define Helper Functions

def get_customer_info(id):
    return CUSTOMERS[id-1]

def cal_distance(loc1, loc2):
    return 100 * math.sqrt((loc2[1] - loc1[1])**2 + (loc2[0] - loc1[0])**2)

def add_customer_info(lat, long, demand):
    new_cus_id = len(CUSTOMERS) + 1
    new_cus_info = {'id':new_cus_id,
                    'loc':(lat,long),
                    'demand':demand}
    return CUSTOMERS.append(new_cus_info)

def add_vehicle_info(capacity, cost):
    uppercase_alphabet = string.ascii_uppercase
    new_vehicle_type = uppercase_alphabet[len(VEHICLES)]
    new_vehicle_info = {'type': new_vehicle_type,
                        'capacity':capacity,
                        'cost per km':cost}
    return VEHICLES.append(new_vehicle_info)

def get_vehicle_info(type):
    curr_vehicle_idx = string.ascii_uppercase.index(type)
    return VEHICLES[curr_vehicle_idx] 

In [40]:
# Add Customers

add_customer_info(4.3555,113.9777,5)
add_customer_info(4.3976,114.0049,8)
add_customer_info(4.3163,114.0764,3)
add_customer_info(4.3184,113.9932,6)
add_customer_info(4.4024,113.9896,5)
add_customer_info(4.4142,114.0127,8)
add_customer_info(4.4804,114.0734,3)
add_customer_info(4.3818,114.2034,6)
add_customer_info(4.4935,114.1928,5)
add_customer_info(4.4932,114.1322,8)

CUSTOMERS

[{'id': 1, 'loc': (4.3555, 113.9777), 'demand': 5},
 {'id': 2, 'loc': (4.3976, 114.0049), 'demand': 8},
 {'id': 3, 'loc': (4.3163, 114.0764), 'demand': 3},
 {'id': 4, 'loc': (4.3184, 113.9932), 'demand': 6},
 {'id': 5, 'loc': (4.4024, 113.9896), 'demand': 5},
 {'id': 6, 'loc': (4.4142, 114.0127), 'demand': 8},
 {'id': 7, 'loc': (4.4804, 114.0734), 'demand': 3},
 {'id': 8, 'loc': (4.3818, 114.2034), 'demand': 6},
 {'id': 9, 'loc': (4.4935, 114.1928), 'demand': 5},
 {'id': 10, 'loc': (4.4932, 114.1322), 'demand': 8}]

In [41]:
# Add Vehicles

add_vehicle_info(25,1.2)
add_vehicle_info(30,1.5)
add_vehicle_info(27,1.3)

VEHICLES

[{'type': 'A', 'capacity': 25, 'cost per km': 1.2},
 {'type': 'B', 'capacity': 30, 'cost per km': 1.5},
 {'type': 'C', 'capacity': 27, 'cost per km': 1.3}]

In [42]:
# Vehicle object constructor 

class Vehicle():
    def __init__(self, name, capacity, mileage_cost, route, starting_loc):
        self.name = name
        self.capacity = capacity
        self.mileage_cost = mileage_cost
        self.route = route
        self.starting_loc = starting_loc

        self.calculate()

    def __str__(self):
        return str({
            'type':self.name,
            'route':self.route
        })

    def calculate(self):
        total_demand = 0
        total_distance = 0

        curr_loc = self.starting_loc
        for customer_id in self.route:
            next_customer = get_customer_info(customer_id) 
            # calculate demand
            total_demand += next_customer['demand']

            # calculate distance
            total_distance += cal_distance(curr_loc,next_customer['loc'])

            curr_customer = next_customer
            curr_loc = curr_customer['loc']

        total_distance += cal_distance(curr_loc,self.starting_loc)
        total_cost = self.mileage_cost * total_distance

        # set mandatory properties with new values
        self.distance = total_distance
        self.cost = total_cost
        self.demand = total_demand

        if self.overloaded():
            self.cost += 10000 # bias value

    def overloaded(self):
        return self.demand > self.capacity
    
    def distance_list(self):
        distance_list = []
        curr_loc = self.starting_loc
        for customer_id in self.route:
            next_customer = get_customer_info(customer_id) 

            distance_list.append(cal_distance(curr_loc,next_customer['loc']))

            curr_customer = next_customer
            curr_loc = curr_customer['loc']

        distance_list.append(cal_distance(curr_loc,self.starting_loc))

        return distance_list

In [43]:
# Soltuion object constructor (base solution)

class Solution():
    def __init__(self, vehicles):
        self.vehicles = vehicles

    def __str__(self):
        return str([{
            'type':vehicle.name,
            'route':vehicle.route
            } for vehicle in self.vehicles])

    def fitness(self):
        total_fitness = 0 # fitness = cost
    
        for vehicle in self.vehicles:
            total_fitness += vehicle.cost

        return total_fitness
    
    # for override
    def mate(self, solution2):
        pass

    def cal_distance(self):
        total_distance = 0

        for vehicle in self.vehicles:
            total_distance += vehicle.distance

        return total_distance

    def print(self):

        
        print(f"Total Distance: {self.cal_distance():.2f}KM")
        print(f"Total cost: RM{self.fitness():.2f}")
        print("\n")

        for vehicle in self.vehicles:
            print(f"Vehicle {vehicle.name}:")
            print(f"Round Trip Distance: {vehicle.distance:.2f} KM, Cost: RM{vehicle.cost:.2f}, Demand: {vehicle.demand}, Capacity: {vehicle.capacity}")
            
            distance_list = vehicle.distance_list()
            vehicle_route = vehicle.route
            route_segments = [f"Depot"]

            for i in range(len(vehicle.route)):
                route_segments.append(f"{vehicle_route[i]} ({distance_list[i]:.2f} KM)")
            
            route_segments.append(f"Depot ({distance_list[len(distance_list)-1]:.2f} KM)")
            route_str = " -> ".join(route_segments)

            print(route_str)
            print('\n')

In [44]:
# Solution 1 (inherit Solution)

class Solution1(Solution):
    def mutate(vehicle):
        start, end = sorted(random.sample(range(10), 2))
        sublist = vehicle.route[start:end]
        random.shuffle(sublist)
        vehicle.route[start:end] = sublist
        vehicle.calculate()

    def generate_child_sol(vehicles, inherit_vehicle):
        vehicles_list = []

        curr_idx = 0
        for vehicle in vehicles:
            if curr_idx == inherit_vehicle: vehicles_list.append(vehicle)
            else:
                mutated_vehicle = Vehicle(
                    vehicle.name,
                    vehicle.capacity,
                    vehicle.mileage_cost,
                    vehicle.route.copy(),
                    vehicle.starting_loc
                )
                Solution1.mutate(mutated_vehicle)

                vehicles_list.append(mutated_vehicle)

        return Solution1(vehicles_list)
    
    def mate(self, solution2):
        prob = random.random()
        # Crossover (cross gene exchange, e.g split ratio and routes)

        if prob < 0.45: 
            inherit_vehicle = random.randint(0,len(VEHICLES)-1)
        
            return Solution1.generate_child_sol(self.vehicles,inherit_vehicle)
    
        elif prob < 0.90: 
            inherit_vehicle = random.randint(0,len(VEHICLES)-1)
        
            return Solution1.generate_child_sol(solution2.vehicles,inherit_vehicle)
        else:
        # Mutation (Self gene exchange)
            vehicle_choice = random.choice([self, solution2])
            return Solution1.generate_child_sol(vehicle_choice.vehicles,-1) # -1 to mutate every vehicle

In [45]:
# Solution 2 (inherit Solution)

class Solution2(Solution):
    def get_split_ratio(vehicles):
        split_portion = []
        for vehicle in vehicles:
            split_portion.append(len(vehicle.route))
        return split_portion

    def create_child_sol(split_ratio,combined_route):
        vehicle_type = [vehicle['type'] for vehicle in VEHICLES]

        vehicles = []

        curr_idx = 0
        for type in vehicle_type:
            curr_vehicle = get_vehicle_info(type)

            route = combined_route[:split_ratio[curr_idx]]
            combined_route = combined_route[split_ratio[curr_idx]:]

            vehicles.append(Vehicle(
                type,
                curr_vehicle['capacity'],
                curr_vehicle['cost per km'],
                route,
                DEPOT_LOC
            ))

            curr_idx+=1

        return Solution2(vehicles)
    
    def mutate(split_ratio,combine_route):
        start, end = sorted(random.sample(range(10), 2))
        sublist = combine_route[start:end]
        random.shuffle(sublist)
        combine_route[start:end] = sublist
        
        return Solution2.create_child_sol(split_ratio,combine_route)
    
    def mate(self, solution2):
        self_split_ratio = Solution2.get_split_ratio(self.vehicles)
        self_combined_route = [customer_id for vehicle in self.vehicles for customer_id in vehicle.route]

        solution2_split_ratio = Solution2.get_split_ratio(solution2.vehicles)
        solution2_combined_route = [customer_id for vehicle in solution2.vehicles for customer_id in vehicle.route]

        prob = random.random()
        # Crossover (cross gene exchange, e.g split ratio and routes)
        if prob < 0.45: 
            # inherit split ratio from parent 1 and route from parent 2
            child_solution = Solution2.create_child_sol(self_split_ratio,solution2_combined_route)
        elif prob < 0.90: 
            # inherit split ratio from parent 2 and route from parent 1
            child_solution = Solution2.create_child_sol(solution2_split_ratio,self_combined_route)
        else:
        # Mutation (Self gene exchange)
            child_solution = Solution2.mutate(self_split_ratio,self_combined_route)

        return child_solution  

Note:

Differenece between Solution 1 and 2 is the mating algorithm, which I have tried using 2 different approaches of cross-over.

Cross-over in Solution 1: Randomly choose one parent and inherit (remain) one of the vehicle route, and the route in other vehicle's route will be shuffled.

Cross-over in Solution 2: Consider the route split ratio and also the sequence of the route are the available genes, then create a children that inherit one the gene from one parent and otherwise with probabilty.

In [46]:
# Geneate Dataset (Population)

def split_random_portion(customer_len, vehicle_len):
    random_nums = []
    curr_amount = customer_len

    for i in range(vehicle_len):
        if i+1 ==  vehicle_len:
            random_nums.append(curr_amount)
            break

        random_num = random.randint(1, curr_amount-vehicle_len+1+i)
        curr_amount-=random_num
        random_nums.append(random_num)

    return random_nums

def create_vehicle(type, route):
    info = get_vehicle_info(type)
    
    return Vehicle(
        type,
        info['capacity'],
        info['cost per km'],
        route,
        DEPOT_LOC
    )

def create_solution(solution_type):
    vehicle_type = [vehicle['type'] for vehicle in VEHICLES]
    vehicles = []

    customers_ids = [customer['id'] for customer in CUSTOMERS]
    random.shuffle(customers_ids) # shuffle customer to ensure diversity

    customers_len = len(CUSTOMERS)
    vehicles_len = len(VEHICLES)
    random_nums = split_random_portion(customers_len,vehicles_len)

    curr_idx = 0
    for type in vehicle_type:
        split_portion = random_nums[curr_idx]
        route = customers_ids[:split_portion]

        customers_ids = customers_ids[split_portion:] # remove occupied route from list

        vehicle = create_vehicle(type,route)
        vehicles.append(vehicle)
        
        curr_idx+=1

    if solution_type == 1:
        return Solution1(vehicles)
    elif solution_type == 2:
        return Solution2(vehicles)

In [47]:
# Create population

POPULATION_SIZE = 200

population = []

first_best_solution = None
for _ in range(POPULATION_SIZE):
    solution = create_solution(2)
    population.append(solution)
    if first_best_solution == None:
        first_best_solution = solution
    elif solution.fitness() < first_best_solution.fitness():
        first_best_solution = solution
    print(solution)

[{'type': 'A', 'route': [4]}, {'type': 'B', 'route': [9, 1, 8, 7]}, {'type': 'C', 'route': [3, 6, 5, 10, 2]}]
[{'type': 'A', 'route': [8, 9, 7, 1]}, {'type': 'B', 'route': [4, 3, 6, 5]}, {'type': 'C', 'route': [2, 10]}]
[{'type': 'A', 'route': [3, 6, 2, 10, 7, 4, 9]}, {'type': 'B', 'route': [8]}, {'type': 'C', 'route': [1, 5]}]
[{'type': 'A', 'route': [3, 2]}, {'type': 'B', 'route': [6, 1, 5]}, {'type': 'C', 'route': [10, 8, 9, 4, 7]}]
[{'type': 'A', 'route': [2, 8, 5, 7, 1]}, {'type': 'B', 'route': [9, 6, 4, 10]}, {'type': 'C', 'route': [3]}]
[{'type': 'A', 'route': [10, 8, 4]}, {'type': 'B', 'route': [5]}, {'type': 'C', 'route': [1, 9, 6, 7, 3, 2]}]
[{'type': 'A', 'route': [2, 6, 1]}, {'type': 'B', 'route': [5, 7]}, {'type': 'C', 'route': [9, 3, 8, 10, 4]}]
[{'type': 'A', 'route': [9, 1, 4, 10]}, {'type': 'B', 'route': [8, 5, 2, 3, 6]}, {'type': 'C', 'route': [7]}]
[{'type': 'A', 'route': [7, 9, 3, 10, 5, 2, 8]}, {'type': 'B', 'route': [1]}, {'type': 'C', 'route': [4, 6]}]
[{'type': 

In [48]:
print("\n<<<< BEST SOLUTION (First Generated Population)>>>>\n")

first_best_solution.print()


<<<< BEST SOLUTION (First Generated Population)>>>>

Total Distance: 127.47KM
Total cost: RM167.73


Vehicle A:
Round Trip Distance: 45.80 KM, Cost: RM54.96, Demand: 17, Capacity: 25
Depot -> 2 (9.07 KM) -> 3 (10.83 KM) -> 8 (14.29 KM) -> Depot (11.61 KM)


Vehicle B:
Round Trip Distance: 33.02 KM, Cost: RM49.52, Demand: 16, Capacity: 30
Depot -> 7 (6.51 KM) -> 9 (12.01 KM) -> 10 (6.06 KM) -> Depot (8.44 KM)


Vehicle C:
Round Trip Distance: 48.65 KM, Cost: RM63.25, Demand: 24, Capacity: 27
Depot -> 5 (10.48 KM) -> 4 (8.41 KM) -> 6 (9.78 KM) -> 1 (6.83 KM) -> Depot (13.15 KM)




In [53]:
# define task function to run GA algo

GENERATION_SIZE = 10

def task(id):
    global population

    population_copy = population.copy()

    for i in range(GENERATION_SIZE):
        population_copy = sorted(population_copy, key = lambda solution:solution.fitness())
        new_generation = []

        # include 10% of the fittest solution in the next generation
        s = int((10*POPULATION_SIZE)/100) 
        new_generation.extend(population_copy[:s])

        s = int((90*POPULATION_SIZE)/100)
        for _ in range(s):
            # only consider choosing among 50% fittest solution as parents,
            # to produce offsprings
            parent1 = random.choice(population_copy[:int(POPULATION_SIZE/2)])
            parent2 = random.choice(population_copy[:int(POPULATION_SIZE/2)])
            child = parent1.mate(parent2)
            new_generation.append(child)

        population_copy = new_generation

        #print(f"Generation {i+1}: Best fitness score (Lowest Total Cost) - RM {(population_copy[0].fitness()):.2f}")

    #print_sol_info(population[0],GENERATION_SIZE)
    print(f"[Task {id+1}] Best fitness score (Lowest Total Cost) - RM {(population_copy[0].fitness()):.2f}")

    return population_copy[0]

In [54]:
# Kinda simulate multi Threading
# Where n number of GAs will run simultaneously, to find the best out of the best
best_solution = None
for i in range(100):
    solution = task(i)
    if best_solution == None:
        best_solution = solution
    elif solution.fitness() < best_solution.fitness():
        best_solution = solution

print(f"\n<<<< BEST SOLUTION (Out of {i+1} Tasks)>>>>\n")

best_solution.print()

[Task 1] Best fitness score (Lowest Total Cost) - RM 133.71
[Task 2] Best fitness score (Lowest Total Cost) - RM 141.67
[Task 3] Best fitness score (Lowest Total Cost) - RM 149.82
[Task 4] Best fitness score (Lowest Total Cost) - RM 153.97
[Task 5] Best fitness score (Lowest Total Cost) - RM 143.22
[Task 6] Best fitness score (Lowest Total Cost) - RM 142.13
[Task 7] Best fitness score (Lowest Total Cost) - RM 152.85
[Task 8] Best fitness score (Lowest Total Cost) - RM 149.44
[Task 9] Best fitness score (Lowest Total Cost) - RM 154.54
[Task 10] Best fitness score (Lowest Total Cost) - RM 147.89
[Task 11] Best fitness score (Lowest Total Cost) - RM 132.34
[Task 12] Best fitness score (Lowest Total Cost) - RM 143.22
[Task 13] Best fitness score (Lowest Total Cost) - RM 150.03
[Task 14] Best fitness score (Lowest Total Cost) - RM 149.61
[Task 15] Best fitness score (Lowest Total Cost) - RM 132.04
[Task 16] Best fitness score (Lowest Total Cost) - RM 136.97
[Task 17] Best fitness score (Low

In [ ]:
# This is task function that apply a while loop to find the best solution of a cost threshold

GENERATION_SIZE = 1000
COST_THRESHOLD = 120

def task():
    global population

    population_copy = population.copy()

    lowest_cost_solution = None

    curr_idx = 0
    while True:
        population_copy = sorted(population_copy, key = lambda solution:solution.fitness())
        new_generation = []

        # include 10% of the fittest solution in the next generation
        s = int((10*POPULATION_SIZE)/100) 
        new_generation.extend(population_copy[:s])

        s = int((90*POPULATION_SIZE)/100)
        for _ in range(s):
            # only consider choosing among 50% fittest solution as parents,
            # to produce offsprings
            parent1 = random.choice(population_copy[:int(POPULATION_SIZE/2)])
            parent2 = random.choice(population_copy[:int(POPULATION_SIZE/2)])
            child = parent1.mate(parent2)
            new_generation.append(child)

        population_copy = new_generation

        lowest_cost_solution = population_copy[0]

        print(f"Generation {curr_idx+1}: Best fitness score (Lowest Total Cost) - RM {(lowest_cost_solution.fitness()):.2f}")
        curr_idx+=1

        if lowest_cost_solution.fitness() <= COST_THRESHOLD:
            break 

    #print_sol_info(population[0],GENERATION_SIZE)

task()

Note: this function is not suitable due to the tasks generated might greater than the threshold infinitely, hence, cant terminate